In [1]:
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow import keras

2022-12-28 03:53:37.221799: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/xilinx/xrt/lib:/usr/lib:/usr/lib/x86_64-linux-gnu
2022-12-28 03:53:37.221814: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
data = np.random.rand(200,1024,768,3)
test=np.random.rand(200,256,192,8)
train=data
#test=np.random.rand(100,64,48,192)

In [3]:
input1 = tf.keras.Input(shape=(1024,768,3))
#x1 = tf.keras.layers.ZeroPadding2D(padding=(2))(input1)
x1 = tf.keras.layers.Conv2D(8,5,strides=(2,2),padding='same',dilation_rate=(1, 1),groups=1,activation=None,use_bias=True)(input1)
x2 = tf.keras.layers.Conv2D(8,3,strides=(2,2),padding='same',dilation_rate=(1, 1),groups=1,activation=None,use_bias=True)(x1)
x3 = tf.keras.layers.Conv2D(8,3,strides=(1,1),padding='same',dilation_rate=(1, 1),groups=1,activation=None,use_bias=True)(x2)
x4 = tf.keras.layers.Conv2D(192,5,strides=(2,2),padding='same',dilation_rate=(1, 1),groups=1,activation=None,use_bias=True)(x3)
x5 = tf.keras.layers.Conv2D(192,5,strides=(2,2),padding='same',dilation_rate=(1, 1),groups=1,activation=None,use_bias=True)(x4)
model1 = keras.Model(inputs=input1, outputs=x3)

2022-12-28 03:54:00.213695: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/xilinx/xrt/lib:/usr/lib:/usr/lib/x86_64-linux-gnu
2022-12-28 03:54:00.213710: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-12-28 03:54:00.213719: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (binod-OptiPlex-5090): /proc/driver/nvidia/version does not exist
2022-12-28 03:54:00.214000: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [4]:
x3

<KerasTensor: shape=(None, 256, 192, 8) dtype=float32 (created by layer 'conv2d_2')>

In [5]:
x_train, x_test, y_train, y_test=train_test_split(train, test, test_size=0.1)

In [6]:
model1.compile(optimizer='adagrad',loss="MeanAbsoluteError",metrics="MeanAbsoluteError")
model1.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1024, 768, 3)]    0         
                                                                 
 conv2d (Conv2D)             (None, 512, 384, 8)       608       
                                                                 
 conv2d_1 (Conv2D)           (None, 256, 192, 8)       584       
                                                                 
 conv2d_2 (Conv2D)           (None, 256, 192, 8)       584       
                                                                 
Total params: 1,776
Trainable params: 1,776
Non-trainable params: 0
_________________________________________________________________


In [7]:
checkpoint = tf.keras.callbacks.ModelCheckpoint('project.h5', monitor='val_loss',
save_best_only=True, verbose=2)
model1.fit(x_train,y_train,batch_size=1,epochs=2,validation_split=(1/9),callbacks=[checkpoint])

2022-12-28 03:54:11.196165: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 1509949440 exceeds 10% of free system memory.


Epoch 1/2
160/160 [==============================] - ETA: 0s - loss: 0.3818 - mean_absolute_error: 0.3818
Epoch 1: val_loss improved from inf to 0.29831, saving model to project.h5
160/160 [==============================] - 14s 66ms/step - loss: 0.3818 - mean_absolute_error: 0.3818 - val_loss: 0.2983 - val_mean_absolute_error: 0.2983
Epoch 2/2
160/160 [==============================] - ETA: 0s - loss: 0.2953 - mean_absolute_error: 0.2953
Epoch 2: val_loss improved from 0.29831 to 0.29376, saving model to project.h5
160/160 [==============================] - 10s 61ms/step - loss: 0.2953 - mean_absolute_error: 0.2953 - val_loss: 0.2938 - val_mean_absolute_error: 0.2938


In [8]:
model1.get_weights()[0].dtype

dtype('float32')

In [9]:
from tensorflow_model_optimization.quantization.keras import vitis_quantize

In [10]:
quantizer = vitis_quantize.VitisQuantizer(model1)
quantized_model = quantizer.quantize_model(calib_dataset = train, weight_bit=8, activation_bit=8)

[VAI INFO] Update activation_bit: 8
[VAI INFO] Update weight_bit: 8
[VAI INFO] Start CrossLayerEqualization...
10/10 [==============================] - 0s 28ms/step
[VAI INFO] CrossLayerEqualization Done.
[VAI INFO] Start Quantize Calibration...


2022-12-28 03:55:01.345978: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 1887436800 exceeds 10% of free system memory.


7/7 [==============================] - 31s 2s/step
[VAI INFO] Quantize Calibration Done.
[VAI INFO] Start Post-Quant Model Refinement...
[VAI INFO] Start Quantize Position Ajustment...
[VAI INFO] Quantize Position Ajustment Done.
[VAI INFO] Post-Quant Model Refninement Done.
[VAI INFO] Start Model Finalization...
[VAI INFO] Model Finalization Done.
[VAI INFO] Quantization Finished.


In [11]:
quantized_model.compile(loss="MeanAbsoluteError",metrics="MeanAbsoluteError")

score = quantized_model.evaluate(x_test, y_test,  verbose=0, batch_size=1)
print(score)

[0.2938533425331116, 0.2938533425331116]


In [12]:
quantized_model.save('custom_dpu_0.h5')

In [14]:
!vai_c_tensorflow2 \
    --model ./custom_dpu_0.h5 \
    --arch /opt/vitis_ai/compiler/arch/DPUCZDX8G/ZCU104/arch.json \
    --output_dir . \
    --net_name custom_dpu_compiled_0

**************************************************
* VITIS_AI Compilation - Xilinx Inc.
**************************************************
[INFO] Namespace(batchsize=1, inputs_shape=None, layout='NHWC', model_files=['./custom_dpu_0.h5'], model_type='tensorflow2', named_inputs_shape=None, out_filename='/tmp/custom_dpu_compiled_0_DPUCZDX8G_ISA1_B4096_org.xmodel', proto=None)
[INFO] tensorflow2 model: /workspace/custom_dpu_0.h5
[INFO] keras version: 2.8.0
[INFO] Tensorflow Keras model type: functional
[INFO] generate xmodel     :100%|█| 8/8 [00:00<00:00, 3399.98it/s]              
[INFO] dump xmodel: /tmp/custom_dpu_compiled_0_DPUCZDX8G_ISA1_B4096_org.xmodel
[UNILOG][INFO] Compile mode: dpu
[UNILOG][INFO] Debug mode: function
[UNILOG][INFO] Target architecture: DPUCZDX8G_ISA1_B4096
[UNILOG][INFO] Graph name: model, with op num: 20
[UNILOG][INFO] Begin to compile...
[UNILOG][INFO] Total device subgraph number 3, DPU subgraph number 1
[UNILOG][INFO] Compile done.
[UNILOG][INFO] The meta jso